Step 0: Importing Relevant Packages (0%)

In [15]:
# Web Scraping Libraries
from bs4 import BeautifulSoup  # for scraping
import requests  # for sending HTTP requests
# Data Processing Libraries
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Step 1: Crawl a Real World Data Set (2%)

In [20]:
url = "https://www.worldometers.info/coronavirus/#main_table"  # Web address of data
response = requests.get(url)  # Performing a get request for the data at the url location
soup_object = BeautifulSoup(response.content)  # Parsing the request response to create a soup_object that can be searched for a table
covid_table = soup_object.find("table")  # Searching the soup_object for a table

with open("Live_Covid_Data.csv", "w", newline="") as csv_file:  # Opening a CSV file to be written into
    writer = csv.writer(csv_file)  # Defining the writer as a csv writer that writes to the opened file

    for row in table.find_all("tr"):  # Iterate through the rows of the table, "tr" points to a row in the table
        cells = row.find_all("td")  # Get the cells in the row, "td" points to a cell in the row

        if cells:  # If there are cells...
            values = [cell.text for cell in cells]  # Values is a list of values for each cell in the row of cells
            writer.writerow(values)  # Write the values to the CSV


Step 2: Data Preparation and Cleaning (5%)

In [57]:
# Creating list of column names
col_names = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Critical Cases",
             "Total Cases / 1M People", "Total Deaths / 1M People", "Total Tests", "Total Tests / 1M People", "Population", "Continent", "One Case / n People",
             "One Death / n People", "One Test / n People", "New Cases / 1M People", "New Deaths / 1M People", "Active Cases / 1M People"]

# Reading the csv to a dataframe with the column names specified, encoding with "latin-1" to get around "utf-8" decoding issue.
df = pd.read_csv("Live_Covid_Data.csv", names=col_names, header=None, encoding="latin-1")
df.head(15)

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Critical Cases,Total Cases / 1M People,...,Total Tests,Total Tests / 1M People,Population,Continent,One Case / n People,One Death / n People,One Test / n People,New Cases / 1M People,New Deaths / 1M People,Active Cases / 1M People
NaN,\nNorth America\n,"121,153,860",20,"1,577,299",NaN,"116,021,050","2,529","3,555,511","9,185",NaN,...,NaN,NaN,NaN,North America,\n,NaN,NaN,NaN,NaN,NaN
NaN,\nAsia\n,"206,269,228","359,815","1,510,515",614.0,"193,256,698","97,077","11,502,015","10,966",NaN,...,NaN,NaN,NaN,Asia,\n,NaN,NaN,NaN,NaN,NaN
NaN,\nEurope\n,"242,566,203","21,183","1,979,053",146.0,"236,644,243","150,556","3,942,907","7,899",NaN,...,NaN,NaN,NaN,Europe,\n,NaN,NaN,NaN,NaN,NaN
NaN,\nSouth America\n,"66,709,623","5,622","1,341,029",5.0,"64,212,906","17,278","1,155,688","10,347",NaN,...,NaN,NaN,NaN,South America,\n,NaN,NaN,NaN,NaN,NaN
NaN,\nOceania\n,"13,580,074","2,117","23,274",NaN,"13,232,661",NaN,"324,139",160,NaN,...,NaN,NaN,NaN,Australia/Oceania,\n,NaN,NaN,NaN,NaN,NaN
NaN,\nAfrica\n,"12,738,671",154,"258,338",1.0,"12,036,221",49,"444,112",843,NaN,...,NaN,NaN,NaN,Africa,\n,NaN,NaN,NaN,NaN,NaN
NaN,\n\n,721,NaN,15,NaN,706,NaN,0,0,NaN,...,NaN,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN
NaN,World,"663,018,380","388,911","6,689,523",766.0,"635,404,485","267,489","20,924,372","39,400","85,059",...,NaN,NaN,NaN,All,\n,NaN,NaN,NaN,NaN,NaN
1.0,USA,"102,307,321",NaN,"1,116,365",NaN,"99,330,862",NaN,"1,860,094","4,002","305,573",...,"1,151,242,966","3,438,545","334,805,269",North America,3,300,0.0,NaN,NaN,"5,556"
2.0,India,"44,678,763",NaN,"530,696",NaN,"44,143,483",NaN,"4,584",698,"31,763",...,"909,993,187","646,931","1,406,631,776",Asia,31,"2,651",2.0,NaN,NaN,3


Step 3: Exploratory Data Analysis (8%)

Step 4a: Question 1 (10%)

Step 4b: Question 2 (10%)

Step 4c: Question 3 (10%)

Step 5: Summary and Conclusion (5%)

Write summary/conc in here